## Demo of filter


This notebook demonstrates the use of the Kalman filter.

## Setup 


First lets quickly set up some bits

In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
import sys
# Hacky way to add higher directory to python modules path. 
# Means that I dont have to make src/ a proper python package
try:
    sys.path.remove("../src") 
except:
    pass
sys.path.append("../src") 

## 1. Canonical example 

This is effectively the same steps that are taken by `main.py`


In [15]:
from system_parameters import SystemParameters
from pulsars import Pulsars
from synthetic_data import SyntheticData
from model import PhaseModel
from kalman_filter import KalmanFilter
from priors import bilby_priors_dict

import numpy as np 



#Setup and create some synthetic data
P    = SystemParameters()    # User input system parameters. Can also be read from config file
PTA  = Pulsars(P)                       # All pulsar-related quantities
data = SyntheticData(PTA,P)             # Given the system parameters and the PTA configuration, create some synthetic data

# #Define the model to be used by the Kalman Filter
# model = PhaseModel(P,PTA)

# #Initialise the Kalman filter
# KF = KalmanFilter(model,data.phi_measured,PTA)

# #Run the KF with the correct parameters.
# #We get the correct parameters via Bilby dictionary, looking towards when we will run this with nested sampling
# init_parameters,optimal_parameters_dict = bilby_priors_dict(PTA,P,set_state_parameters_as_known=True,set_measurement_parameters_as_known=True)
# optimal_parameters                      = optimal_parameters_dict.sample(1)    
# model_likelihood                        = KF.likelihood(optimal_parameters)

    


INFO:root:Welcome to the Slow Kalman Filter
INFO:root:Random seed is 1234


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/t1/miniconda3/envs/KalmanGW/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3398, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/bx/dsrq957n43bckqdm1vkk78yc0000gn/T/ipykernel_43553/1310309289.py", line 15, in <cell line: 15>
    data = SyntheticData(PTA,P)             # Given the system parameters and the PTA configuration, create some synthetic data
  File "/Users/t1/github_repos/SlowKalmanFilter/notebooks/../src/synthetic_data.py", line 68, in __init__
    GW = gw_measurement_equation(Ω=P.Ω,
  File "/Users/t1/github_repos/SlowKalmanFilter/notebooks/../src/gravitational_waves.py", line 69, in gw_measurement_equation
    for k in range(H):
TypeError: only integer scalar arrays can be converted to a scalar index

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/t1/miniconda3/envs/KalmanGW/lib/python3.10/site-